In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# # Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create a Percent Cloudiness Heatmap
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Create a Wind Speed Heatmap
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Atuona,PF,2021-11-09 20:08:10,-9.8000,-139.0333,78.35,76,2,16.73
10,10,Rikitea,PF,2021-11-09 20:08:11,-23.1203,-134.9692,75.29,77,84,20.09
13,13,Tomatlan,MX,2021-11-09 20:08:12,19.9333,-105.2500,89.67,45,61,4.70
21,21,Coihaique,CL,2021-11-09 20:08:14,-45.5752,-72.0662,75.81,33,20,8.05
24,24,Mahebourg,MU,2021-11-09 20:08:15,-20.4081,57.7000,75.49,78,40,5.75
...,...,...,...,...,...,...,...,...,...,...
550,550,Cururupu,BR,2021-11-09 20:21:46,-1.8283,-44.8683,78.85,85,91,4.97
551,551,Mufumbwe,ZM,2021-11-09 20:22:46,-13.6833,24.8000,75.38,32,41,2.84
553,553,Saint-Georges,GD,2021-11-09 20:22:47,12.0564,-61.7485,85.68,66,20,12.66
557,557,San Policarpo,PH,2021-11-09 20:22:47,12.1791,125.5072,80.76,84,99,12.01


In [11]:
preferred_cities_df.count()

City_ID       164
City          164
Country       164
Date          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,78.35,-9.8000,-139.0333,
10,Rikitea,PF,75.29,-23.1203,-134.9692,
13,Tomatlan,MX,89.67,19.9333,-105.2500,
21,Coihaique,CL,75.81,-45.5752,-72.0662,
24,Mahebourg,MU,75.49,-20.4081,57.7000,
26,Arlit,NE,84.04,18.7369,7.3853,
27,Jutai,BR,84.15,-5.1833,-68.9000,
28,Hobyo,SO,79.86,5.3505,48.5268,
29,Butaritari,KI,82.18,3.0707,172.7902,
34,Vaini,TO,77.63,-21.2000,-175.2000,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
# Grab the first hotel from the results and store the name.
hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,78.35,-9.8000,-139.0333,Villa Enata
10,Rikitea,PF,75.29,-23.1203,-134.9692,People ThankYou
13,Tomatlan,MX,89.67,19.9333,-105.2500,El Salvial
21,Coihaique,CL,75.81,-45.5752,-72.0662,Diego De Almagro Coyhaique
24,Mahebourg,MU,75.49,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
26,Arlit,NE,84.04,18.7369,7.3853,Case de Passage De la Prefecture
27,Jutai,BR,84.15,-5.1833,-68.9000,
28,Hobyo,SO,79.86,5.3505,48.5268,Halane Hotel and Restaurent
29,Butaritari,KI,82.18,3.0707,172.7902,Isles Sunset Lodge
34,Vaini,TO,77.63,-21.2000,-175.2000,Keleti Beach Resort


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))